# Azure AI Foundry Customer Service Agent Demo

This notebook demonstrates how to create and use a customer service agent using Azure AI Foundry Agent Service with our ARTAgent-style builder.

## Key Features:
- **No manual schemas required** - Azure AI Foundry auto-generates from function signatures and docstrings
- **Best practices implementation** - Following Azure AI Foundry documentation guidelines
- **Professional tool descriptions** - Clear triggers and examples for each tool
- **JSON-safe outputs** - All tools return properly formatted JSON

## Setup and Imports

In [1]:
# 📂 Setup Working Directory for ARTAgent Framework Access
import logging
import os

# Configure logging to track directory changes
logging.basicConfig(level=logging.INFO)

# Navigate to the project root directory
# This ensures we can import ARTAgent framework modules properly
try:
    # Move up two directories from samples/hello_world/ to project root
    os.chdir("../../")
    
    # Allow override via environment variable for different setups
    target_directory = os.getenv(
        "TARGET_DIRECTORY", os.getcwd()
    )  # Use environment variable if available
    
    # Verify the target directory exists before changing
    if os.path.exists(target_directory):
        os.chdir(target_directory)
        print(f"✅ Changed directory to: {os.getcwd()}")
        logging.info(f"Successfully changed directory to: {os.getcwd()}")
    else:
        print(f"❌ Directory does not exist: {target_directory}")
        logging.error(f"Directory does not exist: {target_directory}")
        
except Exception as e:
    print(f"❌ Error changing directory: {e}")
    logging.exception(f"An error occurred while changing directory: {e}")

# Verify we're in the correct location
print(f"📁 Current working directory: {os.getcwd()}")
print(f"📋 Contents: {', '.join(os.listdir('.')[:10])}...")

INFO:root:Successfully changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent


✅ Changed directory to: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📁 Current working directory: c:\Users\pablosal\Desktop\gbb-ai-audio-agent
📋 Contents: .azure, .devcontainer, .env, .env.aoai_pool, .env.sample, .files, .git, .github, .gitignore, .pre-commit-config.yaml...


In [ ]:
import os
import sys
from pathlib import Path

# Add current directory to path for imports
sys.path.append('.')

# Check for Azure AI Foundry environment variables
endpoint = os.getenv("AZURE_AI_FOUNDRY_ENDPOINT")
print(f"Azure AI Foundry Endpoint: {'✅ Configured' if endpoint else '❌ Not found - set AZURE_AI_FOUNDRY_ENDPOINT'}")

if endpoint:
    print(f"Endpoint: {endpoint}")
else:
    print("\n💡 To run this demo with real Azure AI Foundry:")
    print("   export AZURE_AI_FOUNDRY_URL='https://your-project.cognitiveservices.azure.com'")
    print("   or set it in your .env file")

Azure AI Foundry Endpoint: ✅ Configured
Endpoint: https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice


## Test Tool Registry and Functions

First, let's verify our tools are working properly with the new best practices format.

In [4]:
from samples.hello_world.foundryagents.tool_store import tool_registry

# Validate tool registry
print("🛠️  Tool Registry Validation:")
validation_result = tool_registry.validate_tool_registry()
print(f"Status: {'✅ Passed' if validation_result else '❌ Failed'}")

# Show available tools
print("\n📋 Available Tools:")
for tool_name in tool_registry.TOOL_REGISTRY.keys():
    tool_func = tool_registry.get_tool_function(tool_name)
    # Extract first line of docstring for summary
    doc_summary = tool_func.__doc__.split('\n')[1].strip() if tool_func.__doc__ else "No description"
    print(f"   • {tool_name}: {doc_summary}")

INFO:tool_registry:✅ Tool registry validation passed for 4 tools


🛠️  Tool Registry Validation:
Status: ✅ Passed

📋 Available Tools:
   • check_order_status: Check the current status and details of a customer order.
   • search_knowledge_base: Search the company knowledge base for policies, procedures, and product information.
   • create_support_ticket: Create a new support ticket for customer issues that require follow-up or investigation.
   • escalate_to_human: Escalate a customer issue to a human support agent for immediate personal assistance.


## Test Individual Tools

Let's test each tool to see their enhanced outputs with proper type hints and detailed descriptions.

In [5]:
import json

# Test check_order_status
print("🧪 Testing check_order_status:")
order_tool = tool_registry.get_tool_function('check_order_status')
order_result = order_tool('ORD-DEMO-123')
print(json.dumps(order_result, indent=2))

print("\n" + "="*50)

# Test search_knowledge_base  
print("🧪 Testing search_knowledge_base:")
kb_tool = tool_registry.get_tool_function('search_knowledge_base')
kb_result = kb_tool('return policy')
print(json.dumps(kb_result, indent=2))

INFO:customer_support_tools:Checking order status for order ID: ORD-DEMO-123
INFO:customer_support_tools:Searching knowledge base for: return policy
INFO:customer_support_tools:Searching knowledge base for: return policy


🧪 Testing check_order_status:
{
  "order_id": "ORD-DEMO-123",
  "status": "In Transit",
  "estimated_delivery": "2025-09-05",
  "tracking_number": "TRKORD-DEMO-1232025",
  "items": [
    "Wireless Headphones",
    "Phone Case"
  ],
  "total": "$89.99",
  "shipping_carrier": "FedEx",
  "last_update": "Package left distribution center"
}

🧪 Testing search_knowledge_base:
{
  "query": "return policy",
  "topic": "Return",
  "information": "You can return items within 30 days of purchase. Items must be in original condition with tags attached. Refunds processed within 5-7 business days.",
  "confidence": 0.95,
  "source": "Company Knowledge Base"
}


In [6]:
# Test create_support_ticket
print("🧪 Testing create_support_ticket:")
ticket_tool = tool_registry.get_tool_function('create_support_ticket')
ticket_result = ticket_tool('customer@example.com', 'My order arrived damaged', 'high')
print(json.dumps(ticket_result, indent=2))

print("\n" + "="*50)

# Test escalate_to_human
print("🧪 Testing escalate_to_human:")
escalate_tool = tool_registry.get_tool_function('escalate_to_human')
escalate_result = escalate_tool('TKT-123456', 'Customer is frustrated and wants to speak to a manager')
print(json.dumps(escalate_result, indent=2))

INFO:customer_support_tools:Creating support ticket for: customer@example.com
INFO:customer_support_tools:Escalating ticket TKT-123456 to human agent
INFO:customer_support_tools:Escalating ticket TKT-123456 to human agent


🧪 Testing create_support_ticket:
{
  "ticket_id": "TKT-173513",
  "customer_email": "customer@example.com",
  "issue_description": "My order arrived damaged",
  "priority": "high",
  "status": "Open",
  "created_date": "2025-09-02",
  "estimated_response": "4 hours",
  "assigned_team": "Customer Support",
  "next_steps": "Our support team will review and respond via email with detailed assistance"
}

🧪 Testing escalate_to_human:
{
  "ticket_id": "TKT-123456",
  "escalation_reason": "Customer is frustrated and wants to speak to a manager",
  "escalated_to": "Senior Customer Support Manager",
  "escalation_time": "2025-09-02 10:30:00",
  "priority": "High",
  "expected_response": "Within 2 hours",
  "status": "Escalated - Human Agent Assigned",
  "contact_method": "Phone call priority, then email",
  "queue_position": 1,
  "message": "A senior human agent will contact you personally within 2 hours to resolve this issue with the attention it deserves."
}


## Agent Configuration Preview

Let's examine our agent configuration to see the enhanced instructions following Azure AI Foundry best practices.

In [7]:
import yaml

# Load and display agent configuration
yaml_path = "samples/hello_world/foundryagents/agent_store/customer_service_agent.yaml"
with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

print("🤖 Agent Configuration:")
print(f"   Name: {config['agent']['name']}")
print(f"   Model: {config['model']['deployment_id']}")
print(f"   Tools: {len(config['tools'])} configured")
print(f"   Tools: {', '.join(config['tools'])}")

print("\n📝 Instructions Preview:")
instructions = config['agent']['instructions']
lines = instructions.split('\n')
for i, line in enumerate(lines[:15]):  # Show first 15 lines
    if line.strip():
        print(f"   {line}")
if len(lines) > 15:
    print("   ... (and more)")

🤖 Agent Configuration:
   Name: CustomerServiceAgent
   Model: gpt-4o
   Tools: 4 configured
   Tools: check_order_status, search_knowledge_base, create_support_ticket, escalate_to_human

📝 Instructions Preview:
   You are a professional customer service agent for an e-commerce company. Your primary objective is to help customers resolve their inquiries quickly, accurately, and with a positive experience.
   ## Your Responsibilities:
   - Assist customers with order-related questions using real-time order data
   - Provide accurate information from the company knowledge base
   - Create support tickets for complex issues requiring follow-up
   - Escalate frustrated customers or complex issues to human agents
   - Always be friendly, professional, and empathetic
   ## Available Tools and When to Use Them:
   **check_order_status tool**
   - Use: Get real-time order information including status, tracking, and delivery estimates
   - Trigger: When customers ask about order status, trackin

## Create Agent with Azure AI Foundry Builder

Now let's create the agent using our builder that follows the exact Azure AI Foundry patterns.

In [11]:
# Import the agent builder
from samples.hello_world.foundryagents.agent_builder import AzureFoundryAgentBuilder

# Check if we have Azure credentials
has_azure_creds = bool(os.getenv("AZURE_AI_FOUNDRY_ENDPOINT"))

if has_azure_creds:
    print("✅ Azure AI Foundry credentials detected - creating real agent")
    try:
        # Initialize the builder
        builder = AzureFoundryAgentBuilder()
        
        # Create agent from YAML
        agent_id = builder.create_agent_from_yaml(yaml_path)
        
        print(f"\n🎉 Agent created successfully!")
        print(f"Agent ID: {agent_id}")
        print(f"Agent Name: {config['agent']['name']}")
        print(f"Tools Available: {len(config['tools'])}")
        
    except Exception as e:
        print(f"❌ Agent creation failed: {e}")
        print("\n💡 Make sure you have:")
        print("   - Valid Azure AI Foundry endpoint")
        print("   - Proper authentication (az login or service principal)")
        print("   - Required permissions on the Azure AI Foundry project")
else:
    print("ℹ️  Simulating agent creation (no Azure credentials)")
    print("\n📋 Agent Would Be Created With:")
    print(f"   Name: {config['agent']['name']}")
    print(f"   Model: {config['model']['deployment_id']}")
    print(f"   Tools: {len(config['tools'])} JSON-safe wrapped functions")
    print("   Schema Generation: Automatic from type hints and docstrings")
    print("   Instructions: Enhanced with tool triggers and examples")
    
    # Show what the function signatures look like
    print("\n🔍 Tool Signatures (Auto-Schema Generation):")
    for tool_name in config['tools']:
        tool_func = tool_registry.get_tool_function(tool_name)
        import inspect
        sig = inspect.signature(tool_func)
        print(f"   {tool_name}{sig} -> {tool_func.__annotations__.get('return', 'Any')}")

✅ Azure AI Foundry credentials detected - creating real agent



INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
[2025-09-02 19:45:12,331] INFO - foundry_agent_builder: 🔗 Foundry Agent Builder connecting to: https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS
[2025-09-02 19:45:12,331] INFO - foundry_agent_builder: 🔗 Foundry Agent Builder connecting to: https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice
INFO:foundry_agent_builder:🔗 Foundry Agent Builder connecting to: https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice
[2025-09-02 19:45:12,343]INFO:foundry_agent_builder:🔗 Foundry Agent Builder connecting to: https://poc-ai-agents-voice-resource.services.ai.azure.com/api/projects/poc-ai-agents-voice
[2025-09-02 19:45:12,343


🎉 Agent created successfully!
Agent ID: asst_Kp4exd80NINFuraHyWOftsuR

Agent ID: asst_Kp4exd80NINFuraHyWOftsuR
Agent Name: CustomerServiceAgent
Tools Available: 4
Agent Name: CustomerServiceAgent
Tools Available: 4


## JSON-Safe Wrapper Demonstration

Azure AI Foundry requires all tool outputs to be JSON strings. Let's see how our `json_safe_wrapper` works.

In [12]:
from samples.hello_world.foundryagents.agent_builder import json_safe_wrapper

# Get a raw tool function
raw_tool = tool_registry.get_tool_function('check_order_status')

# Wrap it for Azure AI Foundry
wrapped_tool = json_safe_wrapper(raw_tool)

print("🔧 Raw Tool Output:")
raw_result = raw_tool('ORD-12345')
print(f"Type: {type(raw_result)}")
print(f"Content: {raw_result}")

print("\n🎁 JSON-Safe Wrapped Output:")
wrapped_result = wrapped_tool('ORD-12345')
print(f"Type: {type(wrapped_result)}")
print(f"Content: {wrapped_result}")

print("\n✅ This JSON string is what Azure AI Foundry receives and can parse properly!")

🔧 Raw Tool Output:



INFO:customer_support_tools:Checking order status for order ID: ORD-12345


Type: <class 'dict'>
Content: {'order_id': 'ORD-12345', 'status': 'In Transit', 'estimated_delivery': '2025-09-05', 'tracking_number': 'TRKORD-123452025', 'items': ['Wireless Headphones', 'Phone Case'], 'total': '$89.99', 'shipping_carrier': 'FedEx', 'last_update': 'Package left distribution center'}

🎁 JSON-Safe Wrapped Output:

Content: {'order_id': 'ORD-12345', 'status': 'In Transit', 'estimated_delivery': '2025-09-05', 'tracking_number': 'TRKORD-123452025', 'items': ['Wireless Headphones', 'Phone Case'], 'total': '$89.99', 'shipping_carrier': 'FedEx', 'last_update': 'Package left distribution center'}

🎁 JSON-Safe Wrapped Output:


INFO:customer_support_tools:Checking order status for order ID: ORD-12345


Type: <class 'str'>

Content: {"order_id": "ORD-12345", "status": "In Transit", "estimated_delivery": "2025-09-05", "tracking_number": "TRKORD-123452025", "items": ["Wireless Headphones", "Phone Case"], "total": "$89.99", "shipping_carrier": "FedEx", "last_update": "Package left distribution center"}

✅ This JSON string is what Azure AI Foundry receives and can parse properly!
Content: {"order_id": "ORD-12345", "status": "In Transit", "estimated_delivery": "2025-09-05", "tracking_number": "TRKORD-123452025", "items": ["Wireless Headphones", "Phone Case"], "total": "$89.99", "shipping_carrier": "FedEx", "last_update": "Package left distribution center"}

✅ This JSON string is what Azure AI Foundry receives and can parse properly!
